<a href="https://colab.research.google.com/github/bogdanjianu150/Mask_RCNN/blob/main/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 



In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

!pip install tensorflow==1.15.0
!pip install keras==2.2.5
!pip install h5py==2.10.0


Found existing installation: Keras 2.2.5
Uninstalling Keras-2.2.5:
  Successfully uninstalled Keras-2.2.5
Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Successfully uninstalled Keras-Preprocessing-1.1.2
Found existing installation: tensorflow 1.15.0
Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0
Found existing installation: h5py 2.10.0
Uninstalling h5py-2.10.0:
  Successfully uninstalled h5py-2.10.0
  Using cached tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl (412.3 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached h5py-3.4.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.


  Using cached Keras-2.2.5-py2.py3-none-any.whl (336 kB)


  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
  Attempting uninstall: h5py
    Found existing installation: h5py 3.4.0
    Uninstalling h5py-3.4.0:
      Successfully uninstalled h5py-3.4.0


In [ ]:
pip install 'h5py==2.10.0' --force-reinstall

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 15.7 MB 182 kB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.4.0
    Uninstalling h5py-3.4.0:
      Successfully uninstalled h5py-3.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.13.1 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible

In [110]:
from imgaug import augmenters as iaa
from drive.MyDrive.Mask_RCNN.mrcnn.config import Config
from drive.MyDrive.Mask_RCNN.mrcnn import model as modellib
from drive.MyDrive.Mask_RCNN.mrcnn import visualize
from drive.MyDrive.Mask_RCNN.mrcnn import utils
from imutils import paths
import numpy as np
import argparse
import imutils
import random
import cv2
import os
import json
from google.colab.patches import cv2_imshow


In [111]:
!nvidia-smi


Wed Sep  8 17:33:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    61W / 149W |   4222MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [112]:
dataset_path = os.path.abspath("/content/drive/MyDrive/trashnet")
images_path = os.path.abspath("/content/drive/MyDrive/trashnet/dataset")
masks_path = os.path.sep.join([dataset_path, "masks.json"])



training_split = 0.75

image_paths = sorted(list(paths.list_images(images_path)))
idxs = list(range(0, len(image_paths) ))
random.seed(42)
random.shuffle(idxs)
i = int(len(idxs) * training_split)
train_idxs = idxs[:i]
val_idxs = idxs[i:]




In [113]:
class_names = {1 : "cardboard", 2 : "glass", 3: "paper", 4: "plastic", 5: "metal", 6: "trash"}
#class_names ={1: "trash"}

coco_path = "/content/drive/MyDrive/mask_rcnn_coco.h5"

LOGS_AND_MODEL_DIR = "logs"


In [114]:
class TrashConfig(Config):
  NAME = 'trash'
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1
  STEPS_PER_EPOCH = len(train_idxs) // (IMAGES_PER_GPU * GPU_COUNT)
  NUM_CLASSES = len(class_names) + 1

In [115]:
class TrashInferenceConfig(TrashConfig):
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1
  DETECTION_MIN_CONFIDENCE = 0.9
  

In [116]:
class TrashDataset(utils.Dataset):
  def __init__(self, imagePaths, masksPath, classNames, width = 1024):
    super().__init__(self)
    self.imagePaths = imagePaths
    self.classNames = classNames
    self.width = width
    self.masks = self.load_mask_data(masksPath)

  def load_mask_data(self, masksPath):
   masks = json.loads(open(masksPath).read())
  
   return masks

  def load_trash(self, idxs):
    for (class_id, label) in  self.classNames.items():
      self.add_class("trash", class_id, label)


    image_path = os.listdir(self.imagePaths)

    for i in image_path :
      #image_path = os.listdir(self.imagePaths)
      #filename = image_path.split(os.path.sep)[-1]
      
      k = self.imagePaths + '/' + i
      filename = k.split(os.path.sep)[-1]
      image = cv2.imread(k)
      self.add_image("trash", image_id = filename, path = k)
  
  def load_image(self, image_id):
   p = self.image_info[image_id]["path"]
   image = cv2.imread(p)
   image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
   image = imutils.resize(image, width = self.width)
   return image

  def load_mask(self, image_id):
   image_info = self.image_info[image_id]
   if image_info["source"] != "/content/drive/MyDrive/trashnet/dataset":
    return super(self.__class__, self).load_mask(image_id)
   info = self.image_info[imageID]
   m2 = self.m[info["id"]]

   masks = np.zeros((info["height"], info["width"], len(m2["regions"])), dtype = "uint8")

   for (i, region) in enumerate(m2["regions"]) :
    region_mask = np.zeros(masks2.shape[:2], dtype = "uint8")
    sa = region["shape_attributes"]
    ra = region["region_attributes"]
    ratio = info["width"] / float(info["orig_width"])
    cX = int(sa["cx"] * ratio)
    cY = int(sa["cy"] * ratio)
    r = int(sa["r"] * ratio)

    

    cv2.circle(region_mask, (cX, cY), r, 1, -1)
    
    masks[:, :, i] = region_mask

   return (masks2.astype("bool"), np.ones((masks.shape[-1], ), dtype ="int32"))
      

In [ ]:
train_dataset = TrashDataset(images_path, masks_path, class_names)
train_dataset.load_trash(train_idxs)
train_dataset.prepare()

In [ ]:
val_dataset = TrashDataset(images_path, masks_path, class_names)
val_dataset.load_trash(val_idxs)
val_dataset.prepare()

In [ ]:
config = TrashConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [ ]:
model = modellib.MaskRCNN(mode = "training", config = config, model_dir = LOGS_AND_MODEL_DIR)
model.load_weights(coco_path, by_name = True, exclude = ["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

model.train(train_dataset, val_dataset, epochs = 20, layers = "heads", learning_rate = config.LEARNING_RATE)
model.train(train_dataset, val_dataset, epochs = 40, layers = "all", learning_rate = config.LEARNING_RATE / 10)


Starting at epoch 0. LR=0.001

Checkpoint Path: logs/trash20210908T1726/mask_rcnn_trash_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistri

Epoch 1/20


RuntimeError: ignored

In [ ]:
config = TrashInferenceConfig()
model = modellib.MaskRCNN(mode = "inference", config = config, model_dir = logs_and_model_dir)

weights = args["weights"] if args[weights] else model.find_last()
model.load_weigths(weights, by_name = True )


image = cv2.imread(args["image"])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = imutils.resize(image, width = 1024)

r = model.detect([image], verbose = 1)[0]

for i in range(0, r["rois"].shape[0]):
  mask = r["masks"][:, :, i]
  image = visualize.apply_mask(image, mask, (1.0, 0.0, 0.0), alpha = 0.5)
  image = visualize.draw_box(image, r["rois"][i], [1.0, 0.0, 0.0])
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

  for i in range(0, len(r["scores"])):
   (startY, startX, endY, endX) = r["rois"][i]
   class_id = r["class_ids"][i]
   label = class_names[class_id]
   score = r["scores"][i]

   text = "{}: {:.4f}".format(label, score)
   y = startY - 10 if startY - 10 > 10 else startY + 10
   cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
      
image = imutils.resize(image, width = 512)
cv2.imshow("Output", image)
cv2.waitKey(0)

/content/drive/MyDrive/trashnet/dataset/plastic125.jpg
/content/drive/MyDrive/trashnet/dataset/plastic126.jpg
/content/drive/MyDrive/trashnet/dataset/plastic124.jpg
/content/drive/MyDrive/trashnet/dataset/plastic127.jpg
/content/drive/MyDrive/trashnet/dataset/plastic128.jpg
/content/drive/MyDrive/trashnet/dataset/plastic129.jpg
/content/drive/MyDrive/trashnet/dataset/plastic130.jpg
/content/drive/MyDrive/trashnet/dataset/plastic13.jpg
/content/drive/MyDrive/trashnet/dataset/plastic131.jpg
/content/drive/MyDrive/trashnet/dataset/plastic133.jpg
/content/drive/MyDrive/trashnet/dataset/plastic132.jpg
/content/drive/MyDrive/trashnet/dataset/plastic134.jpg
/content/drive/MyDrive/trashnet/dataset/plastic135.jpg
/content/drive/MyDrive/trashnet/dataset/plastic136.jpg
/content/drive/MyDrive/trashnet/dataset/plastic137.jpg
/content/drive/MyDrive/trashnet/dataset/plastic139.jpg
/content/drive/MyDrive/trashnet/dataset/plastic138.jpg
/content/drive/MyDrive/trashnet/dataset/plastic14.jpg
/content/dri

KeyboardInterrupt: ignored